第一部分 数据预处理及分析


常用库的导入

In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

数据载入，csv文件用pandas中的read_csv

In [2]:
data = pd.read_csv("./data.csv",encoding='gbk')
data.head()

检查缺失值

In [5]:
# 查看数据信息，看看有无缺失值
data.info()

# 删除了对预测结果几乎无任何相关性的特征
delete_columns = ["Unnamed: 0", "trade_no", "bank_card_no", "first_transaction_time", "source", 
                  "id_name", "latest_query_time", "loans_latest_time","student_feature"]
source = data.drop(delete_columns, axis=1)
source.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 90 columns):
Unnamed: 0                                    4754 non-null int64
custid                                        4754 non-null int64
trade_no                                      4754 non-null object
bank_card_no                                  4754 non-null object
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility

,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,loans_max_limit,loans_avg_limit,consfin_credit_limit,consfin_credibility,consfin_org_count_current,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day
0,2791858,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,...,2900.0,1688.0,1200.0,75.0,1.0,2.0,1200.0,1200.0,12.0,18.0
1,534047,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,...,3500.0,1758.0,15100.0,80.0,5.0,6.0,22800.0,9360.0,4.0,2.0
2,2849787,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,...,1600.0,1250.0,4200.0,87.0,1.0,1.0,4200.0,4200.0,2.0,6.0
3,1809708,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,...,3200.0,1541.0,16300.0,80.0,5.0,5.0,30000.0,12180.0,2.0,4.0
4,2499829,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,...,2300.0,1630.0,8300.0,79.0,2.0,2.0,8400.0,8250.0,22.0,120.0


In [6]:
source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 81 columns):
custid                                        4754 non-null int64
low_volume_percent                            4752 non-null float64
middle_volume_percent                         4752 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4751 non-null float64
trans_activity_month                          4752 non-null float64
trans_activity_day                            4752 non-null float64
transd_mcc                                    4752 non-null float64
trans_days_interval_filter                    4746 non-null float64
trans_days_interval                           4752 non-null float64
regional_mobility                             4752 non-null float64
repayment_capability                          4754 non-null int64
is_high_user                                  4754 non-null int64
number_of_trans_f

缺失值处理
缺失值较多的“student feature”直接删除。对于其它含有缺失值的特征用出现频率最高的值来补充

In [7]:
nan = ['low_volume_percent', 'middle_volume_percent', 'trans_amount_increase_rate_lately', 
       'trans_activity_month','trans_activity_day', 'transd_mcc', 'trans_days_interval_filter',
       'trans_days_interval', 'regional_mobility',
       'number_of_trans_from_2011',
       'historical_trans_day', 'rank_trad_1_month',
       'avg_consume_less_12_valid_month',
       'top_trans_count_last_1_month',
       'avg_price_top_last_12_valid_month', 'reg_preference_for_trad',
       'trans_top_time_last_1_month', 'trans_top_time_last_6_month',
       'consume_top_time_last_1_month', 'consume_top_time_last_6_month',
       'cross_consume_count_last_1_month',
       'trans_fail_top_count_enum_last_1_month',
       'trans_fail_top_count_enum_last_6_month',
       'trans_fail_top_count_enum_last_12_month',
       'consume_mini_time_last_1_month',
       'max_consume_count_later_6_month',
       'railway_consume_count_last_12_month',
       'jewelry_consume_count_last_6_month',
       'first_transaction_day', 'trans_day_last_12_month',
       'apply_score', 'apply_credibility', 'query_org_count',
       'query_finance_count', 'query_cash_count', 'query_sum_count',
       'latest_one_month_apply',
       'latest_three_month_apply', 'latest_six_month_apply', 'loans_score',
       'loans_credibility_behavior', 'loans_count', 'loans_settle_count',
       'loans_overdue_count', 'loans_org_count_behavior',
       'consfin_org_count_behavior', 'loans_cash_count',
       'latest_one_month_loan', 'latest_three_month_loan',
       'latest_six_month_loan', 'history_suc_fee', 'history_fail_fee',
       'latest_one_month_suc', 'latest_one_month_fail', 'loans_long_time',
       'loans_credit_limit', 'loans_credibility_limit',
       'loans_org_count_current', 'loans_product_count', 'loans_max_limit',
       'loans_avg_limit', 'consfin_credit_limit', 'consfin_credibility',
       'consfin_org_count_current', 'consfin_product_count',
       'consfin_max_limit', 'consfin_avg_limit', 'latest_query_day',
       'loans_latest_day']
for key in nan:
    fill_value = source[key].value_counts().index[0]
    source[key].fillna(fill_value, inplace=True)

In [8]:
source.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4754 entries, 0 to 4753
Data columns (total 81 columns):
custid                                        4754 non-null int64
low_volume_percent                            4754 non-null float64
middle_volume_percent                         4754 non-null float64
take_amount_in_later_12_month_highest         4754 non-null int64
trans_amount_increase_rate_lately             4754 non-null float64
trans_activity_month                          4754 non-null float64
trans_activity_day                            4754 non-null float64
transd_mcc                                    4754 non-null float64
trans_days_interval_filter                    4754 non-null float64
trans_days_interval                           4754 non-null float64
regional_mobility                             4754 non-null float64
repayment_capability                          4754 non-null int64
is_high_user                                  4754 non-null int64
number_of_trans_f

缺失值补充完毕

离散值数值化：对“reg_preference_for_trad”特征进行编码

In [9]:
source = pd.get_dummies(source,columns=['reg_preference_for_trad'])
source.head()

,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,reg_preference_for_trad_一线城市,reg_preference_for_trad_三线城市,reg_preference_for_trad_二线城市,reg_preference_for_trad_其他城市,reg_preference_for_trad_境外
0,2791858,0.01,0.99,0,0.90,0.55,0.313,17.0,27.0,26.0,...,2.0,1200.0,1200.0,12.0,18.0,1,0,0,0,0
1,534047,0.02,0.94,2000,1.28,1.00,0.458,19.0,30.0,14.0,...,6.0,22800.0,9360.0,4.0,2.0,1,0,0,0,0
2,2849787,0.04,0.96,0,1.00,1.00,0.114,13.0,68.0,22.0,...,1.0,4200.0,4200.0,2.0,6.0,1,0,0,0,0
3,1809708,0.00,0.96,2000,0.13,0.57,0.777,22.0,14.0,6.0,...,5.0,30000.0,12180.0,2.0,4.0,0,1,0,0,0
4,2499829,0.01,0.99,0,0.46,1.00,0.175,13.0,66.0,42.0,...,2.0,8400.0,8250.0,22.0,120.0,1,0,0,0,0


特征分析（统计学与绘图）
每一行是一个贷款用户的数据，现在特征有81列，具体的意思可以根据data_description来查询，我们要预测的是是否逾期，即“status”。共4754条记录，训练集有4754*70%≈3328条记录，测试集有4754-3328=1426条记录，数据量还是很小的。

In [12]:
# 相关性协方差表,corr()函数,返回结果接近0说明无相关性,大于0说明是正相关,小于0是负相关.
source_corr = source.drop('status',axis=1).corr()
source_corr

,custid,low_volume_percent,middle_volume_percent,take_amount_in_later_12_month_highest,trans_amount_increase_rate_lately,trans_activity_month,trans_activity_day,transd_mcc,trans_days_interval_filter,trans_days_interval,...,consfin_product_count,consfin_max_limit,consfin_avg_limit,latest_query_day,loans_latest_day,reg_preference_for_trad_一线城市,reg_preference_for_trad_三线城市,reg_preference_for_trad_二线城市,reg_preference_for_trad_其他城市,reg_preference_for_trad_境外
custid,1.000000,0.031365,0.090722,-0.030282,-0.001308,-0.118559,-0.216221,-0.147451,0.143875,0.200035,...,-0.131605,-0.112232,-0.066924,0.096330,0.095289,-0.066667,0.073612,-0.010920,0.007554,0.005392
low_volume_percent,0.031365,1.000000,-0.151563,-0.065983,0.005335,0.023771,-0.093618,-0.095791,0.048524,0.062467,...,-0.064594,-0.095161,-0.086292,0.066013,0.046436,-0.070706,0.019866,0.006937,0.093107,0.113042
middle_volume_percent,0.090722,-0.151563,1.000000,-0.135159,0.001600,-0.031478,-0.055726,-0.094700,0.013999,-0.000488,...,-0.012859,-0.163180,-0.155657,-0.001598,-0.001282,0.151123,-0.078432,-0.105982,-0.024312,-0.099457
take_amount_in_later_12_month_highest,-0.030282,-0.065983,-0.135159,1.000000,-0.006692,0.060774,0.112756,0.181402,-0.054753,-0.053239,...,0.025227,0.051455,0.046994,-0.000320,-0.003356,-0.089791,0.076141,0.078842,-0.005659,-0.022870
trans_amount_increase_rate_lately,-0.001308,0.005335,0.001600,-0.006692,1.000000,0.015372,-0.003044,-0.011616,0.013239,0.000526,...,-0.018687,0.006806,0.039888,-0.001426,0.027410,0.006196,-0.004427,-0.002147,-0.000510,-0.003329
trans_activity_month,-0.118559,0.023771,-0.031478,0.060774,0.015372,1.000000,0.062941,0.069304,-0.092140,-0.151713,...,0.006195,-0.021915,-0.038389,-0.023341,-0.012697,-0.038102,0.005253,0.044518,0.027340,0.039517
trans_activity_day,-0.216221,-0.093618,-0.055726,0.112756,-0.003044,0.062941,1.000000,0.674366,-0.489215,-0.541823,...,0.535051,0.213051,0.095111,-0.182617,-0.269946,0.050882,-0.016728,-0.052499,-0.028666,-0.037423
transd_mcc,-0.147451,-0.095791,-0.094700,0.181402,-0.011616,0.069304,0.674366,1.000000,-0.402482,-0.447310,...,0.368309,0.147382,0.072660,-0.116395,-0.185623,0.083506,-0.015945,-0.031854,-0.034071,-0.141863
trans_days_interval_filter,0.143875,0.048524,0.013999,-0.054753,0.013239,-0.092140,-0.489215,-0.402482,1.000000,0.680321,...,-0.275457,-0.106308,-0.046492,0.065705,0.149810,-0.162434,0.075695,0.054683,0.008300,0.185848
trans_days_interval,0.200035,0.062467,-0.000488,-0.053239,0.000526,-0.151713,-0.541823,-0.447310,0.680321,1.000000,...,-0.276888,-0.100546,-0.027529,0.098946,0.165088,-0.087746,0.056218,0.038460,0.004408,0.055507


无关特征删除

In [13]:
# 计算相关性
cor = source.corr()
corr = np.abs(cor["status"]).sort_values(ascending=False)
corr

status                                     1.000000
trans_fail_top_count_enum_last_1_month     0.333339
history_fail_fee                           0.306273
loans_overdue_count                        0.266961
latest_one_month_fail                      0.248837
loans_score                                0.245092
apply_score                                0.232480
rank_trad_1_month                          0.146643
trans_fail_top_count_enum_last_6_month     0.134436
latest_one_month_suc                       0.131443
trans_fail_top_count_enum_last_12_month    0.118328
top_trans_count_last_1_month               0.117299
consfin_avg_limit                          0.081569
trans_day_last_12_month                    0.078974
consfin_credit_limit                       0.076057
avg_price_top_last_12_valid_month          0.071259
trans_top_time_last_1_month                0.067794
consume_top_time_last_1_month              0.065610
consfin_max_limit                          0.061551
latest_six_m

In [14]:
# 选择相关程度在0.04以上的特征组合起来作为新的特征
index = list(corr[corr>0.05].index)
del index[0]
# 特征工程
new_feature_data = source[index]
new_feature_data.head()

,trans_fail_top_count_enum_last_1_month,history_fail_fee,loans_overdue_count,latest_one_month_fail,loans_score,apply_score,rank_trad_1_month,trans_fail_top_count_enum_last_6_month,latest_one_month_suc,trans_fail_top_count_enum_last_12_month,...,trans_top_time_last_1_month,consume_top_time_last_1_month,consfin_max_limit,latest_six_month_loan,latest_query_day,query_cash_count,low_volume_percent,custid,latest_three_month_loan,avg_price_last_12_month
0,1.0,7.0,2.0,0.0,552.0,583.0,0.40,2.0,1.0,2.0,...,4.0,4.0,1200.0,13.0,12.0,6.0,0.01,2791858,1.0,1020
1,0.0,4.0,0.0,1.0,635.0,653.0,0.35,3.0,2.0,3.0,...,13.0,13.0,22800.0,8.0,4.0,2.0,0.02,534047,2.0,1210
2,3.0,2.0,0.0,1.0,633.0,654.0,1.00,6.0,1.0,6.0,...,0.0,0.0,4200.0,4.0,2.0,5.0,0.04,2849787,2.0,570
3,1.0,26.0,4.0,0.0,542.0,595.0,0.15,8.0,2.0,8.0,...,6.0,6.0,30000.0,34.0,2.0,4.0,0.00,1809708,4.0,1290
4,3.0,25.0,6.0,0.0,479.0,541.0,0.65,3.0,0.0,3.0,...,0.0,0.0,8400.0,10.0,22.0,4.0,0.01,2499829,0.0,1110


标准化

In [16]:
# 标准化
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
new_feature_data_std = std.fit_transform(new_feature_data)
new_feature_data_std = pd.DataFrame(new_feature_data_std, columns = new_feature_data.columns)
new_feature_data_std.head()

,trans_fail_top_count_enum_last_1_month,history_fail_fee,loans_overdue_count,latest_one_month_fail,loans_score,apply_score,rank_trad_1_month,trans_fail_top_count_enum_last_6_month,latest_one_month_suc,trans_fail_top_count_enum_last_12_month,...,trans_top_time_last_1_month,consume_top_time_last_1_month,consfin_max_limit,latest_six_month_loan,latest_query_day,query_cash_count,low_volume_percent,custid,latest_three_month_loan,avg_price_last_12_month
0,-0.341011,-0.389299,-0.053076,-0.325010,0.197137,0.169757,-0.292324,-0.566526,-0.077578,-0.677859,...,-0.586704,-0.556073,-1.096180,-0.028515,-0.329940,0.912736,-0.284251,1.064536,-0.481900,-0.283482
1,-0.865152,-0.510926,-0.697573,-0.060665,1.575817,1.564789,-0.481787,-0.341825,0.447042,-0.467428,...,1.104626,1.092723,0.459607,-0.493400,-0.549139,-0.654592,-0.043372,-1.118767,-0.189051,-0.035373
2,0.707270,-0.592011,-0.697573,-0.060665,1.542596,1.584718,1.981228,0.332277,-0.077578,0.163865,...,-1.338406,-1.288870,-0.880098,-0.865308,-0.603939,0.520904,0.438384,1.120553,-0.189051,-0.871108
3,-0.341011,0.381009,0.591422,-0.325010,0.031030,0.408905,-1.239638,0.781679,0.447042,0.584728,...,-0.210853,-0.189674,0.978203,1.924002,-0.603939,0.129072,-0.525129,0.114797,0.396645,0.069093
4,0.707270,0.340467,1.235920,-0.325010,-1.015438,-0.667262,0.654989,-0.341825,-0.602199,-0.467428,...,-1.338406,-1.288870,-0.577584,-0.307446,-0.055941,0.129072,-0.284251,0.782144,-0.774748,-0.165957


In [17]:
# 清洗之后的数据
final_data = pd.concat([new_feature_data_std,source["status"]], axis=1)
final_data.head()

,trans_fail_top_count_enum_last_1_month,history_fail_fee,loans_overdue_count,latest_one_month_fail,loans_score,apply_score,rank_trad_1_month,trans_fail_top_count_enum_last_6_month,latest_one_month_suc,trans_fail_top_count_enum_last_12_month,...,consume_top_time_last_1_month,consfin_max_limit,latest_six_month_loan,latest_query_day,query_cash_count,low_volume_percent,custid,latest_three_month_loan,avg_price_last_12_month,status
0,-0.341011,-0.389299,-0.053076,-0.325010,0.197137,0.169757,-0.292324,-0.566526,-0.077578,-0.677859,...,-0.556073,-1.096180,-0.028515,-0.329940,0.912736,-0.284251,1.064536,-0.481900,-0.283482,1
1,-0.865152,-0.510926,-0.697573,-0.060665,1.575817,1.564789,-0.481787,-0.341825,0.447042,-0.467428,...,1.092723,0.459607,-0.493400,-0.549139,-0.654592,-0.043372,-1.118767,-0.189051,-0.035373,0
2,0.707270,-0.592011,-0.697573,-0.060665,1.542596,1.584718,1.981228,0.332277,-0.077578,0.163865,...,-1.288870,-0.880098,-0.865308,-0.603939,0.520904,0.438384,1.120553,-0.189051,-0.871108,1
3,-0.341011,0.381009,0.591422,-0.325010,0.031030,0.408905,-1.239638,0.781679,0.447042,0.584728,...,-0.189674,0.978203,1.924002,-0.603939,0.129072,-0.525129,0.114797,0.396645,0.069093,0
4,0.707270,0.340467,1.235920,-0.325010,-1.015438,-0.667262,0.654989,-0.341825,-0.602199,-0.467428,...,-1.288870,-0.577584,-0.307446,-0.055941,0.129072,-0.284251,0.782144,-0.774748,-0.165957,1


In [18]:
# train_test_split
from sklearn.model_selection import train_test_split

X, y = final_data.iloc[:,:-1], final_data.iloc[:,-1]
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2018)

In [19]:
# logisticRegression
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train, y_train)
LR.score(x_test, y_test)

0.7932725998598459

In [20]:
# SVC
from sklearn.svm import SVC

svc = SVC(kernel="rbf", C=1)
svc.fit(x_train, y_train)
svc.score(x_test, y_test)

0.7806587245970568

In [21]:
# 随机森林
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier(n_estimators=500, min_samples_split=2, bootstrap=True, max_depth=4, max_features=6)
RF.fit(x_train, y_train)
RF.score(x_test, y_test)

0.7813594954449895

In [22]:
# ExtraTreesclassifier
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(n_estimators=500, bootstrap=True, n_jobs=-1)
et_clf.fit(x_train, y_train)
et_clf.score(x_test, y_test)

0.7883672039243167

In [25]:
#XGBOOST
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(n_estimators=500,max_depth=4,subsample=1,colsample_btree=1)
xgb_clf.fit(x_train, y_train)
xgb_clf.score(x_test, y_test)

0.767344078486335

In [26]:
#LGBM
from lightgbm import LGBMClassifier
lgb_clf = LGBMClassifier(num_leaves=60,learning_rate=0.05,n_estimators=40)
lgb_clf.fit(x_train, y_train)
lgb_clf.score(x_test, y_test)

0.7715487035739313

In [27]:
#GBDT
from sklearn.ensemble import GradientBoostingClassifier
gbr = GradientBoostingClassifier(n_estimators=3000, max_depth=2, min_samples_split=2, learning_rate=0.1)
gbr.fit(x_train, y_train)
gbr.score(x_test, y_test)

0.7498248072880168

In [28]:
#MLP
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier(activation='relu', solver='adam', alpha=0.0001,max_iter=10000)
NN.fit(x_train, y_train)
NN.score(x_test, y_test)

0.731604765241766

In [30]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)
lda.score(x_test, y_test)

0.7813594954449895

In [31]:
#AdaBoost
from sklearn.ensemble import AdaBoostClassifier
adb = AdaBoostClassifier(n_estimators = 500)
adb.fit(x_train, y_train)
adb.score(x_test, y_test)

0.7575332866152769

In [32]:
#Naive Bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x_train, y_train)
gnb.score(x_test, y_test)

0.7540294323756132

综上，对于该数据，最好的模型是逻辑回归，分数为0.7932725998598459。